# Deploy streamlit app into a cloud run (gcp)

**Following the ideas of:**
- deploy streamlit app in cloud run: https://medium.com/@faizififita1/how-to-deploy-your-streamlit-web-app-to-google-cloud-run-ba776487c5fe
- deploy streamlit app into google app engine: https://dev.to/whitphx/how-to-deploy-streamlit-apps-to-google-app-engine-407o
- deploy a flask app into a cloud run: my previous codes
    - Create Dockerfile (an example Dockerfile can be found at: https://firebase.google.com/docs/hosting/cloud-run?hl=es-419#python)


**Steps:**
- Create a sript with codes
- Create a .env file with environment variables that you don't want to expose
- Define the parameters to deploy
- Run codes that do diferents steps to deploy

## I) INTRODUCTION

These codes could be run in the Google SDK console, as well as run on a notebook.
For a data scientist who is not specialized in devops practices, using a notebook is more intuitive to use

#### Previous steps
- Stop in the root folder of the application (this notebook is created in the root)
- Have scripts created to contain and upload to cloud run

#### Important information
**To run a console command in a Jupyter notebook and the python variables stored in the notebook can also be passed to the command, you must use the peso sign ($) and not use the assignment command (=)**

## II) INITIALIZE READ .ENV WITH ENV VARIABLES

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv # package used in jupyter notebook to read the variables in file .env

""" get env variable from .env """
load_dotenv(find_dotenv())

""" Read env variables and save it as python variable """
WLSACCESSID = os.environ.get("WLSACCESSID", "")
WLSSECRET = os.environ.get("WLSSECRET", "")
LICENSEID = int(os.environ.get("LICENSEID", ""))
PROJECT_GCP = os.environ.get("PROJECT_GCP", "")

## III) DEFINE PARAMETERS TO DEPLOY APP INTO A CLOUD RUN 

### Step 0: Connect to GCP project

In [ ]:
# ! gcloud config set project $PROJECT_GCP

### Step 1: Define parameters

In [ ]:
# PARARAMETERS

# general gcp
REGION = 'us-east1'

# name of the repo in artifact registry where will be saved docker images
NAME_REPO = 'repo-gurobi-optimization-ml-tips'
FORMAT_REPO = 'docker'
DESCRIPTION_REPO = "repo web app with codes of optimization gurobi"

# name of the docker image saved in docker repo in artifact registry
NAME_IMAGE = 'gurobi-optimization-ml-tips'

# name of cloud run where the app web will be located
NAME_CLOUD_RUN = 'gurobi-optimization-ml-tips'

# IV) Upload a docker image with the codes of the app in Artifact Registry

- Artifact registry is the replacement for container registry and recommended by google. The only difference is that the image is saved in this new service and you need to run another command

- **Additionally, every time a new image is uploaded to the artifact registry (rerun the corresponding gcloud command), it receives the latest tag and is the one used to create/update the created cloud run**

- Uploading the image to the artifact Registry requires more steps than uploading it to the container registry

- Cloud build integration documentation with Artifact Registry: https://cloud.google.com/artifact-registry/docs/configure-cloud-build?hl=es-419

### Step 1. Create repository in artifact registry (if it does not exist)
- Unlike container registry which was automatic, in artifact registry you have to create it. **If the repo already exists the gcloud command return an error but doesn't stop de execution of the notebook**

- A repo is created which can have multiple images and each one have different versions

- Documentation: create repo in artifact registry: https://cloud.google.com/artifact-registry/docs/repositories/create-repos#gcloud

In [ ]:
# create repo artifact registry
! gcloud artifacts repositories create $NAME_REPO \
--repository-format $FORMAT_REPO \
--location $REGION \
--description "$DESCRIPTION_REPO" \
--async

### Step 2: Set up a Docker build

It is necessary to create a **yaml** with the configuration to build the docker image in Artifact Registry.

It has the following form

<code>
steps:
- name: 'gcr.io/cloud-builders/docker'
   args: [ 'build', '-t', '${_LOCATION}-docker.pkg.dev/$PROJECT_ID/${_REPOSITORY}/${_IMAGE}', '.' ]
images:
- '${_LOCATION}-docker.pkg.dev/$PROJECT_ID/${_REPOSITORY}/${_IMAGE}'
<code>
    

**This is a GENERIC FILE that can be recycled because it is parameterized to work with any docker repo in the artifact registry**

--> Running the following line of code creates a yaml file with the desired configuration.

Documentation: https://stackabuse.com/reading-and-writing-yaml-to-a-file-in-python/

In [ ]:
import yaml

# create a python diccionary with the content of the yaml cloudbuild generic

dict_python_yaml_cloudbuild = {'steps': [{'name': 'gcr.io/cloud-builders/docker',
   'args': ['build', '-t', '${_LOCATION}-docker.pkg.dev/$PROJECT_ID/${_REPOSITORY}/${_IMAGE}', '.']}],
 'images': ['${_LOCATION}-docker.pkg.dev/$PROJECT_ID/${_REPOSITORY}/${_IMAGE}']}

# save dicctionary in yaml format
with open(r'cloudbuild.yaml', 'w') as file:
    documents = yaml.dump(dict_python_yaml_cloudbuild, file)

### Step 3: Create Dockerfile
Like the previous step, in this case you need to create the Dockerfile to be able to upload the image to the Artifact Registry. Typically, the dockerfile is created manually.

In this example, the content of the dockerfile is defined within a parameterized string, which allows it to be a transversal file and applicable to any deploy in a cloud run

Generally, most of the time, **the Dockerfile does not need to be modified unless you want to change the python vers**ion. So by running the following code you obtain the Dockerfile of the web app

https://prnt.sc/uwBOFChK8QU8

In [ ]:
# CREATE A STRING THAT REPRESENT THE DOCKER FILE

#old version with requirements file inside the script
# string_dockerfile = '''
# FROM python:3.10
# EXPOSE 8080
# WORKDIR /app
# COPY . ./
# RUN pip install streamlit gunicorn
# ENTRYPOINT ["streamlit", "run", "app.py", "--server.port=8080", "--server.address=0.0.0.0"]
# '''


string_dockerfile = '''
FROM python:3.10
EXPOSE 8080
WORKDIR /app
COPY . ./
RUN pip install -r requirements.txt
ENTRYPOINT ["streamlit", "run", "app.py", "--server.port=8080", "--server.address=0.0.0.0"]
''' 

In [ ]:
# guardar dockerfile
with open('Dockerfile', 'w') as file:
    file.write(string_dockerfile)

### Step 4: Containerize (docker image) web app codes using cloud build and upload them to artifact registry
- In this step, a docker image is created with the necessary codes for the web app and then this image is uploaded to Artifact Registry (using as a base the "cloudbuild.yaml" file that calls the "Dockerfile", created in the steps previous)

In [ ]:
##### VERY IMPORTANT NOTATION

# NOTE: The variable names in the gcloud command correspond to the variables defined in the configuration file
#yaml

# NOTE2: to pass the name of the variables (as always) you must use the dollar sign "$" but you must
# to be enclosed in quotes (so that it is understood that it is the variable to be replaced in the configuration yaml)

# NOTE3: it must be double quotes and without spaces to avoid problems

! gcloud builds submit \
    --config=cloudbuild.yaml \
    --substitutions=_LOCATION="$REGION",_REPOSITORY="$NAME_REPO",_IMAGE="$NAME_IMAGE" .

### Step 5: Deploy the artifact registry container image to cloud run
You must run the gloud run deploy command (same as in the container registry) with the only difference being that it changes the location of the image, which is:

  {LOCATION}-docker.pkg.dev/{PROJECT}/{REPOSITORY}/{IMAGE}/
 
 

**IMPORTANT: DUE TO PERMISSIONS ISSUES, THE CLOUD RUN IS CONFIGURED SO THAT ANYONE WITH THE LINK CAN ACCESS**

In [ ]:
#### como setear variables de ambiente en cloud run
#--set-env-vars=PROJECT_GCP=$PROJECT_ID \

In [ ]:
#### try save env varibles cloud run

# ! gcloud run deploy $NAME_CLOUD_RUN \
#     --image $REGION-docker.pkg.dev/$PROJECT_ID/$NAME_REPO/$NAME_IMAGE \
#     --region $REGION \
#     --set-env-vars=PROJECT_GCP=$PROJECT_GCP \
#     --set-env-vars=WLSACCESSID=$WLSACCESSID \
#     --set-env-vars=WLSSECRET=$WLSSECRET \
#     --set-env-vars=LICENSEID=$LICENSEID \
#     --allow-unauthenticated

In [ ]:
# read .env file instead of read env variable saved in the cloud run
! gcloud run deploy $NAME_CLOUD_RUN \
    --image $REGION-docker.pkg.dev/$PROJECT_ID/$NAME_REPO/$NAME_IMAGE \
    --region $REGION \
    --set-env-vars=PROJECT_GCP=$PROJECT_GCP \
    --allow-unauthenticated

In [ ]:
#### I CANT UNDERSTAND WHY IT RETURN A ERROR. THE NAME OF CLOUD RUN IS CORRECT AND THERE IS OTHER ERROR AND GCP SAYS IT IS A CLOUD RUN NAME
NAME_CLOUD_RUN